In [1]:
import streamlit as st
import pickle as pkl

import torch
from sklearn.model_selection import train_test_split

from denoisers.ConditionalUnetDenoiser import ConditionalUnetDenoiser
from denoisers.ConditionalUnetMatrixDenoiser import ConditionalUnetMatrixDenoiser
from utils.graph_utils import get_process_model_reachability_graph_transition_matrix
from utils.pm_utils import discover_dk_process, remove_duplicates_dataset, pad_to_multiple_of_n
from utils.Config import Config
import plotly.express as px
import plotly.graph_objects as go
from dataset.dataset import SaladsDataset
from ddpm.ddpm_multinomial import Diffusion
import os
import json
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
def load_experiment_config(target_dir):
    config_path = os.path.join(target_dir, "cfg.json")
    if os.path.exists(config_path):
        with open(config_path, "r") as f:
            return Config(**json.load(f))
    else:
        st.warning("Configuration file not found.")
        return None


def load_experiment_data_and_model(target_dir, cfg):
    with open(cfg.data_path, "rb") as f:
        base_dataset = pkl.load(f)
    dataset = SaladsDataset(base_dataset['target'], base_dataset['stochastic'])
    train_dataset, test_dataset = train_test_split(dataset, train_size=cfg.train_percent, shuffle=True,
                                                   random_state=cfg.seed)
    dk_process_model, dk_init_marking, dk_final_marking = discover_dk_process(train_dataset, cfg,
                                                                              preprocess=remove_duplicates_dataset)
    diffuser = Diffusion(noise_steps=cfg.num_timesteps, device=cfg.device)
    if cfg.enable_matrix:
        rg_nx, rg_transition_matrix = get_process_model_reachability_graph_transition_matrix(dk_process_model,
                                                                                             dk_init_marking)
        rg_transition_matrix = torch.tensor(rg_transition_matrix, device=cfg.device).float()
        rg_transition_matrix = pad_to_multiple_of_n(rg_transition_matrix)
        denoiser = ConditionalUnetMatrixDenoiser(in_ch=cfg.num_classes, out_ch=cfg.num_classes,
                                                 max_input_dim=dataset.sequence_length,
                                                 transition_dim=rg_transition_matrix.shape[-1],
                                                 device=cfg.device).to(cfg.device).float()
    else:
        rg_transition_matrix = torch.randn((cfg.num_classes, 2, 2)).to(cfg.device)
        denoiser = ConditionalUnetDenoiser(in_ch=cfg.num_classes, out_ch=cfg.num_classes,
                                           max_input_dim=dataset.sequence_length,
                                           device=cfg.device).to(cfg.device).float()
    ckpt_path = os.path.join(target_dir, "best.ckpt")
    denoiser.load_state_dict(torch.load(ckpt_path, map_location=cfg.device)['model_state'])
    final_res_path = os.path.join(target_dir, "final_results.json")
    if os.path.exists(final_res_path):
        with open(final_res_path, "r") as f:
            final_res = json.load(f)
    else:
        st.warning("Final results not found.")

    return (train_dataset, test_dataset, dk_process_model, dk_init_marking, dk_final_marking, rg_transition_matrix,
            diffuser, denoiser, final_res)

In [3]:
target_dir = r"D:\Projects\trace-denoise\pretty_results\50_salads_matrix_no_dropout_new"
cfg = load_experiment_config(target_dir)
cfg.device = "cuda:0"
train_dataset, test_dataset, dk_process_model, dk_init_marking, dk_final_marking, rg_transition_matrix, diffuser, denoiser, final_res = load_experiment_data_and_model(target_dir, cfg)
with open(os.path.join(target_dir, "trace_kinds_train.pkl"), "rb") as f:
    x_train, y_train, x_hat_train = pkl.load(f)
with open(os.path.join(target_dir, "trace_kinds_test.pkl"), "rb") as f:
    x_test, y_test, x_hat_test = pkl.load(f)

RuntimeError: Error(s) in loading state_dict for ConditionalUnetMatrixDenoiser:
	size mismatch for outc_mat.weight: copying a param with shape torch.Size([21, 64, 1, 1]) from checkpoint, the shape in current model is torch.Size([1, 64, 1, 1]).
	size mismatch for outc_mat.bias: copying a param with shape torch.Size([21]) from checkpoint, the shape in current model is torch.Size([1]).